In [1]:
import a3mtools
import a3mtools.examples as examples
%load_ext autoreload
%autoreload 2   

# import an a3m file

In [2]:
msa = a3mtools.MSAa3m.from_a3m_file(examples.a3m_file1)
print(msa)

#9	1
>101
ABCDEFGHI
>ortho1
xxABCDxxEFGZZ
>ortho2
A--DxxE-GHIxxxx
>ortho3
----xxEFGH-



# slicing the alignment

In [4]:
print(msa[2:5])

#3	1
>101
CDE
>ortho1
CDxxE
>ortho2
-DxxE
>ortho3
--xxE



# concatenating alignments

In [5]:
msa2 = a3mtools.MSAa3m.from_a3m_file(examples.a3m_file2)
print(msa2)

#17	1
>101
JKLMNOPQRSTUVW
>ortho1
JKLMNOPQRSTUVWxxxxxx
>ortho2
------PQRSTUVW



In [6]:
print(msa + msa2)

#9,17	1,1
>101	102
ABCDEFGHIJKLMNOPQRSTUVW
>101
ABCDEFGHI--------------
>ortho1
xxABCDxxEFGZZ--------------
>ortho2
A--DxxE-GHIxxxx--------------
>ortho3
----xxEFGH---------------
>102
---------JKLMNOPQRSTUVW
>ortho1
---------JKLMNOPQRSTUVWxxxxxx
>ortho2
---------------PQRSTUVW



In [7]:
print(msa + msa2[2:5] + msa[5:])

#9,3,4	1,1,1
>101	102	103
ABCDEFGHILMNFGHI
>101
ABCDEFGHI-------
>ortho1
xxABCDxxEFGZZ-------
>ortho2
A--DxxE-GHIxxxx-------
>ortho3
----xxEFGH--------
>102
---------LMN----
>ortho1
---------LMN----
>103
------------FGHI
>ortho1
------------FGZZ
>ortho2
-------------GHI
>ortho3
------------FGH-



# saving the alignment to a file

In [8]:
complex_msa = msa + msa2
complex_msa.save("example_complex.a3m")

In [9]:
complex_msa.query.header

'101\t102'

# importing a fasta file

In [21]:
x = a3mtools.import_fasta(a3mtools.examples.fasta_file1)
type(x[0])

a3mtools.backend.sequence_utils.ProteinSequence

In [18]:
[len(i) for i in x]

[15, 15, 15]

In [47]:
famsa = a3mtools.MSAfasta.from_fasta_file(a3mtools.examples.fasta_file1)
print(famsa)

>q
LVT---FLAGCQ---
>a
LVTTTTFL--CQQQQ
>b
LVTTTTFLAGCQQQQ
>c
LVT---FLAGCQQQQ


In [48]:
famsa.print_MSA()

LVT---FLAGCQ---
LVTTTTFL--CQQQQ
LVTTTTFLAGCQQQQ
LVT---FLAGCQQQQ


```
LVT---FLAGCQ---
LVTTTTFL--CQQQQ
LVTTTTFLAGCQQQQ
LVT---FLAGCQQQQ
```
should convert to
```
LVTFLAGCQ
LVTtttFL--CQqqq
LVTtttFLAGCQqqq
LVTFLAGCQQQQ
```

In [49]:
import a3mtools.backend.sequence_utils as utils
from a3mtools import ProteinSequence

In [53]:
query_id = 'q'
aligned_query = famsa.to_dict()[query_id]
unaligned_query, inds = utils.reindex_alignment_str(aligned_query.seq_str)
query = ProteinSequence(query_id, unaligned_query)
new_seqs = []
for pseq in famsa.sequences:
    if pseq.header == query_id:
        continue
    new_seq_str = ''
    for c in range(len(pseq.seq_str)):
        if c in inds:
            new_seq_str += pseq.seq_str[c]
        else:
            if pseq.seq_str[c] != '-':
                new_seq_str += pseq.seq_str[c].lower()
            # else:
            #     new_seq_str += '-'
    new_seqs.append(ProteinSequence(pseq.header, new_seq_str))
info_line = f'#{len(query.seq_str)}\t1'
a3mtools.MSAa3m(info_line, query, new_seqs)

#9	1
>q
LVTFLAGCQ
>a
LVTtttFL--CQqqq
>b
LVTtttFLAGCQqqq
>c
LVTFLAGCQqqq

In [34]:
print(id(famsa.sequences[0].seq_str))
print(id(famsa[2:5].sequences[0].seq_str))

4591888496
4593046064
